# Описание задачи

**Цель**: сделать инструмент подготовки данных для платформы Яндекс.Толока.

In [1]:
import pandas as pd

# прогрессбар
from tqdm.auto import tqdm
tqdm.pandas()

DATASETS_PATH = '../../DATASETS/title'

Считаем датасет:

In [2]:
def read_df():
    df = pd.read_feather(f'{DATASETS_PATH}/total.feather')    
    df = df.set_index('url')  # feather doesn't work with str indices
    
    # parse timing cols ad datetime
    for col in ('post_time', 'parse_time'):
        df[col] = pd.to_datetime(df[col])
    
    return df

df = read_df()

# Задача определения типа заголовка

Нам нужно определять, что за название перед нами, какой у него тип: новость это, инструкция, дайджест, название курса или что-то еще.

Напишем функцию, которая возвращает три файла: n заголовков для эталонной разметки (`gold`), m заголовков для обучения (`silver`), size-n-m для последующего теста или последующего обучения.

In [3]:
dft = df.copy()

# заменяем неразрывные пробелы на обычные
dft.title = dft.title.apply(lambda x: x.replace('\xa0', ' '))

d = {'gold': 1000,
     'silver': 10000}
dfs = dict.fromkeys(d.keys())

for key in dfs:
    dfs[key] = dft[['title']].sample(d[key], random_state=0)
    dft.drop(dfs[key].index, inplace=True)
    dfs[key].to_csv(f'{DATASETS_PATH}/{key}.csv')

dft.reset_index(inplace=True)
dft.to_feather(f'{DATASETS_PATH}/total_exluding_titles.feather')

# Подготовка данных для spacy

Предварительно размеченный набор данных используем для обучения модели. Чтобы сразу обучать модель вместе с NER-разметкой, составим config-файл: